In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
def get_data(column_names):
    train_dataframe = pd.read_csv('./data/housing_train.txt', delim_whitespace=True, header = None)
    test_dataframe = pd.read_csv('./data/housing_test.txt', delim_whitespace=True, header = None)
    train_dataframe.columns = column_names
    test_dataframe.columns = column_names
        
    return train_dataframe, test_dataframe 

In [3]:
column_names = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
train_data, test_data = get_data(column_names)

In [4]:
def normalize(dataset, train_len):
    
    # normalize data using shift/scale
    maxs = dataset.max()
    mins = dataset.min()
    
    for feature in dataset.columns[:-1]:        
        for i, entry in dataset.iterrows():
            dataset.at[i, feature] = (entry[feature] - mins[feature]) / (maxs[feature] - mins[feature])
            
    return dataset.iloc[:train_len], dataset.iloc[train_len:]

In [5]:
full_data = pd.concat([train_data, test_data])
train, test = normalize(full_data, len(train_data))

In [6]:
def get_thresholds(dataset):

    ts = []
    for feature in dataset.columns[:-1]:
        t = []
        for entry in range(len(dataset) - 1):
            t.append((dataset.iloc[entry][feature] + dataset.iloc[entry+1][feature]) / 2)
        ts.append(t)

    return ts

In [26]:
def get_best_split(dataset, thresholds):
    
    best_feature = 'CRIM'
    best_threshold = 0
    max_info_gain = 0
    
    mse_before = get_mse(dataset)
    
    for feature in dataset.columns[:-1]:
        for threshold in range(len(thresholds)):
            left, right = split_data(dataset, feature, threshold)
            
            if len(left) == 0 or len(right) == 0:
                continue
                
            left_mse = get_mse(left)
            right_mse = get_mse(right)
            
#             w = len(left) / len(dataset)
#             mse_after = (w * left_mse) + ((1-w) * right_mse)
            
            mse_after = mse_before - (left_mse + right_mse)
        
            info_gain = mse_before - mse_after

            if max_info_gain <= info_gain:
                max_info_gain = mse_after
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold, max_info_gain

In [27]:
class Terminal:

    def __init__(self, dataset):
        self.prediction = dataset['MEDV'].mean()

    def predict(self):
        return self.prediction

In [28]:
class Node:

    def __init__(self, feature, threshold, left_node, right_node):
        self.feature = feature
        self.threshold = threshold
        self.left_node = left_node
        self.right_node = right_node

In [29]:
def get_mse(dataset):
    
    prediction = dataset['MEDV'].mean()
    errors = []

    for i,entry in dataset.iterrows():
        errors.append(np.square(entry['MEDV'] - prediction))
    
    mse = pd.Series(errors).mean()
    return mse

In [30]:
def split_data(dataset, feature, threshold):    
    
    left = dataset[dataset[feature] < threshold]
    right = dataset[dataset[feature] >= threshold]
    
    return left, right

In [56]:
def build_tree(dataset, depth):

    thresholds = get_thresholds(dataset)    
    best_feature, best_threshold, info_gain = get_best_split(dataset, thresholds)
    
    if info_gain == 0 or depth > 5:
        return Terminal(dataset)
    
    left_data, right_data = split_data(dataset, best_feature, best_threshold)

    left_node = build_tree(left_data, depth+1)
    right_node = build_tree(right_data, depth+1)

    return Node(best_feature, best_threshold, left_node, right_node)

In [57]:
## CREATE REGRESSION TREE
model = build_tree(train, 0)

In [58]:
def regress(root, entry):
    
    if isinstance(root, Terminal):
        return root.predict()
    
    if entry[root.feature] < root.threshold:
        result = regress(root.left_node, entry)
    else:
        result = regress(root.right_node, entry)
        
    return result

In [59]:
def test_model(model, test_data):
    
    predictions = []
    
    for i, entry in test_data.iterrows():
        predictions.append(regress(model, entry))
    
    errors = []
    
    for i,p in enumerate(predictions):
        errors.append(np.square(test_data.iloc[i]['MEDV'] - p))
    
    mse = pd.Series(errors).mean()
    return mse, predictions

In [60]:
final_mse, preds = test_model(model, test)
print('MSE: {}'.format(final_mse))

MSE: 41.39175112628705
